In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch import autograd
from torch.nn import init
import math
from ops import mu_law_encode, one_hot, time_to_batch, batch_to_time, mu_law_decode
from model import WaveNetModel
from audio_reader import AudioReader
import torch.optim as optim
import librosa
import time
import Queue
import json
FILE_PATTERN = r'p([0-9]+)_([0-9]+)\.wav'

In [2]:
with open('./wavenet_params.json', 'r') as config_file:
    wavenet_params = json.load(config_file)

In [3]:
model = WaveNetModel(
                 wavenet_params["batch_size"],
                 wavenet_params["dilations"],
                 wavenet_params["filter_width"],
                 wavenet_params["residual_channels"],
                 wavenet_params["dilation_channels"],
                 wavenet_params["skip_channels"],
                 quantization_channels = wavenet_params["quantization_channels"],
                 use_biases = wavenet_params["use_biases"],
                 scalar_input = wavenet_params["scalar_input"],
                 use_cuda = wavenet_params["use_cuda"],
                 initial_filter_width = wavenet_params["initial_filter_width"],
                 global_condition_channels=None,
                 global_condition_cardinality=None)
if model.use_cuda:
    model = model.cuda()
model.load_state_dict(torch.load('./logdir3/start2.txt'))

In [4]:
waveform = [model.quantization_channels / 2] * (model.receptive_field - 1)
waveform.append(np.random.randint(model.quantization_channels))
to_pred=np.array(waveform)

In [5]:
num_iters = 32000

'''for i, x in enumerate(to_pred):
    if model.use_cuda:    
        loss = model.predict_proba_incremental(autograd.Variable(torch.FloatTensor([x])).cuda())
    else:
        loss = model.predict_proba_incremental(autograd.Variable(torch.FloatTensor([x])))   
'''
start_time = time.time()
for it in range(num_iters):
    model.zero_grad()
    
    to_pred=np.array([waveform[-model.receptive_field:]]).reshape(1,-1)
    if model.use_cuda:    
        loss = model.predict_proba_incremental(autograd.Variable(torch.LongTensor(to_pred)))
    else:
        loss = model.predict_proba_incremental(autograd.Variable(torch.LongTensor(apmlitude)))
    if model.use_cuda:  
        index = np.random.choice(np.arange(model.quantization_channels),p=loss.cpu().data.numpy())
    else:
        index = np.random.choice(np.arange(model.quantization_channels),p=loss.data.numpy())
    #_, index = torch.max(loss,1)
    #index = index.cpu().data.numpy()[0]
    if it % 500 == 0:
        print (it,time.time() - start_time,index)
    waveform.append(index)
    if it == 0:
        print (to_pred[-1])
        print (time.time() - start_time)

(0, ' ', 45)
[128 128 128 ..., 128 128  54]
0.538985013962
(500, ' ', 190)
(1000, ' ', 198)
(1500, ' ', 197)
(2000, ' ', 65)
(2500, ' ', 206)
(3000, ' ', 142)
(3500, ' ', 196)
(4000, ' ', 89)
(4500, ' ', 62)
(5000, ' ', 179)
(5500, ' ', 178)
(6000, ' ', 126)
(6500, ' ', 171)
(7000, ' ', 177)
(7500, ' ', 35)
(8000, ' ', 189)
(8500, ' ', 60)
(9000, ' ', 182)
(9500, ' ', 139)
(10000, ' ', 138)
(10500, ' ', 204)
(11000, ' ', 58)
(11500, ' ', 223)
(12000, ' ', 214)
(12500, ' ', 72)
(13000, ' ', 218)
(13500, ' ', 72)
(14000, ' ', 72)
(14500, ' ', 182)
(15000, ' ', 57)
(15500, ' ', 222)
(16000, ' ', 47)
(16500, ' ', 206)
(17000, ' ', 199)
(17500, ' ', 78)
(18000, ' ', 213)
(18500, ' ', 55)
(19000, ' ', 46)
(19500, ' ', 44)
(20000, ' ', 202)
(20500, ' ', 95)


KeyboardInterrupt: 

In [6]:

to_save = autograd.Variable(torch.FloatTensor(waveform))
to_save = mu_law_decode(to_save,256)
librosa.output.write_wav('new_gen.wav', to_save.data.numpy(), 16000)
